<a href="https://colab.research.google.com/github/stoicelephant/DIDA340/blob/main/Copy_of_Final_Project_Dida_340.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
# %env KAGGLE_USERNAME=jamesgiuf
# %env KAGGLE_KEY=c886c47a2c9510261eb39b5a38d569b9

# !kaggle datasets download -d mouadjaouhari/bitcoin-hourly-ohclv-dataset

# !unzip /content/bitcoin-hourly-ohclv-dataset.zip



env: KAGGLE_USERNAME=jamesgiuf
env: KAGGLE_KEY=c886c47a2c9510261eb39b5a38d569b9
Dataset URL: https://www.kaggle.com/datasets/mouadjaouhari/bitcoin-hourly-ohclv-dataset
License(s): MIT
bitcoin-hourly-ohclv-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/bitcoin-hourly-ohclv-dataset.zip
replace btc_hourly_ohclv_ta.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [159]:
import pandas as pd
import datetime as dt
bitcoin = pd.read_csv('/content/btc_hourly_ohclv_ta.csv')

In [160]:
bitcoin.columns

Index(['UNIX_TIMESTAMP', 'DATETIME', 'OPEN', 'HIGH', 'CLOSE', 'LOW', 'VOLUME',
       'SMA_20', 'EMA_12', 'EMA_26', 'MACD', 'MACD_SIGNAL', 'MACD_DIFF', 'RSI',
       'BB_HIGH', 'BB_LOW', 'BB_WIDTH', 'STOCH_K', 'STOCH_D', 'VOLUME_SMA',
       'MFI', 'ATR', 'PRICE_CHANGE', 'HIGH_LOW_RATIO', 'CLOSE_OPEN_RATIO',
       'VOLATILITY_30D', 'PRICE_VOLATILITY_30D', 'HL_VOLATILITY_30D'],
      dtype='object')

In [161]:
bitcoin['dt'] = pd.to_datetime(bitcoin['DATETIME'])

In [162]:
bitcoin[['DATETIME','OPEN','HIGH','CLOSE','LOW']].head(1)

,DATETIME,OPEN,HIGH,CLOSE,LOW
0,2014-12-15 06:00:00.000,349.78,350.07,349.54,348.81


In [163]:
bitcoin.DATETIME[1][0:7]

'2014-12'

In [164]:
b1512 = bitcoin.loc[bitcoin.DATETIME.str.startswith('2015-12')]

In [165]:
b1512.reset_index(drop=True)
b1512

,UNIX_TIMESTAMP,DATETIME,OPEN,HIGH,CLOSE,LOW,VOLUME,SMA_20,EMA_12,EMA_26,...,VOLUME_SMA,MFI,ATR,PRICE_CHANGE,HIGH_LOW_RATIO,CLOSE_OPEN_RATIO,VOLATILITY_30D,PRICE_VOLATILITY_30D,HL_VOLATILITY_30D,dt
8418,1448928000,2015-12-01 00:00:00.000,378.03,379.28,378.57,376.87,791941.04,376.2890,376.283931,373.989639,...,1.857052e+06,49.776318,3.720053,0.001428,1.006395,1.001428,0.058440,31.113312,0.016339,2015-12-01 00:00:00
8419,1448931600,2015-12-01 01:00:00.000,378.57,378.72,376.29,376.20,705943.21,376.3700,376.284865,374.160036,...,1.844668e+06,45.763894,3.634335,-0.006023,1.006699,0.993977,0.058437,31.108437,0.016337,2015-12-01 01:00:00
8420,1448935200,2015-12-01 02:00:00.000,376.29,378.40,377.00,375.46,749345.02,376.1535,376.394886,374.370404,...,1.757394e+06,47.073934,3.584739,0.001887,1.007830,1.001887,0.058336,31.095033,0.016316,2015-12-01 02:00:00
8421,1448938800,2015-12-01 03:00:00.000,377.00,377.87,377.42,375.75,940507.61,376.0495,376.552596,374.596300,...,1.693427e+06,46.126286,3.480115,0.001114,1.005642,1.001114,0.058335,31.080995,0.016294,2015-12-01 03:00:00
8422,1448942400,2015-12-01 04:00:00.000,377.42,378.30,377.72,376.72,708502.27,376.1295,376.732196,374.827685,...,1.618118e+06,61.736088,3.344392,0.000795,1.004194,1.000795,0.058326,31.070336,0.016291,2015-12-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9157,1451588400,2015-12-31 19:00:00.000,430.77,431.14,430.58,429.99,440238.44,425.0325,426.944057,426.327877,...,1.228743e+06,71.763909,3.090551,-0.000441,1.002674,0.999559,0.034034,31.082005,0.009620,2015-12-31 19:00:00
9158,1451592000,2015-12-31 20:00:00.000,430.58,431.51,430.99,429.83,962480.14,425.3045,427.566510,426.673220,...,1.244267e+06,88.554767,2.989797,0.000952,1.003909,1.000952,0.034030,30.996895,0.009614,2015-12-31 20:00:00
9159,1451595600,2015-12-31 21:00:00.000,430.99,433.54,432.57,430.95,879519.94,425.6640,428.336278,427.110018,...,1.228018e+06,88.394524,2.961240,0.003666,1.006010,1.003666,0.034035,30.912246,0.009615,2015-12-31 21:00:00
9160,1451599200,2015-12-31 22:00:00.000,432.57,432.99,430.98,430.33,788517.51,425.9845,428.743004,427.396683,...,1.242871e+06,83.992750,2.939723,-0.003676,1.006181,0.996324,0.034039,30.829475,0.009611,2015-12-31 22:00:00


In [166]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Candlestick(x=b1512['DATETIME'],
                open=b1512['OPEN'],
                high=b1512['HIGH'],
                low=b1512['LOW'],
                close=b1512['CLOSE'])])

fig.show()

In [167]:
maxt= bitcoin.UNIX_TIMESTAMP.max()
mint= bitcoin.UNIX_TIMESTAMP.min()
week_secs = 60*60*24*7

range_dict = {}
count = 0

while mint <= maxt:
  inputs = [mint, mint+week_secs]
  mint = mint+week_secs
  range_dict[count] = inputs
  count +=1

Now i need to split the df into subframes

In [168]:
df_dict = {}
count = 0

for week in range_dict:
  count +=1
  current_range = range_dict[week]
  lhs = current_range[0]
  rhs = current_range[1]
  new_df =  bitcoin.loc[(bitcoin.UNIX_TIMESTAMP <= rhs) & (bitcoin.UNIX_TIMESTAMP > lhs)]
  df_dict[week] = new_df


In [174]:
num_weeks = len(range_dict) #565

In [171]:
del range

In [216]:
!pip install --upgrade kaleido

In [221]:
import plotly.graph_objects as go
from PIL import Image
import matplotlib.pyplot as plt
import plotly.express as px



num_weeks = len(range_dict)

for i in range(4):
  current_df = df_dict[i]
  fig = go.Figure(data=[go.Candlestick(x=current_df['DATETIME'],
                                       open=current_df['OPEN'],
                                       high=current_df['HIGH'],
                                       low=current_df['LOW'],
                                       close=current_df['CLOSE'])])

  fig.update_yaxes(dtick=20, showgrid=True, gridcolor="lightgray")
  fig.show()
  # fig.write_image("fig1.png")


Still trying to figure out a way on how to standardize thse images